In [180]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf
import os
from tqdm import tqdm
import wave
import contextlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import datasets, layers, models
from sklearn import metrics
from sklearn.metrics import r2_score, confusion_matrix

In [181]:
click = 1

In [182]:
try:
    X = np.load("X.npy")
    Y = np.load("Y.npy")
except:
    click = 0

In [183]:
os.listdir("Training_Files")

['.DS_Store',
 'Beatles',
 'Others',
 'Led Zeppelin',
 'Nirvana',
 'Pink Floyd',
 'Elvis Presley',
 'Beach Boys',
 'Queen',
 'Bob Dylan']

In [184]:
y1, sr1 = librosa.load("/Users/viralchitlangia/Documents/Songs_Audio_Trial/Training_Files/Beatles/ISawHerStandingThereRemastered2009.wav")

In [185]:
librosa.feature.mfcc(y = y1, sr = sr1).shape

(20, 7493)

In [186]:
y2, sr2 = y2, sr2 = librosa.load("/Users/viralchitlangia/Documents/Songs_Audio_Trial/Training_Files/Beatles/WhileMyGuitarGentlyWeepsRemastered2009.wav")

In [187]:
librosa.feature.mfcc(y = y2, sr = sr2).shape

(20, 12275)

In [188]:
len(librosa.feature.mfcc(y = y2, sr = sr2)[0])

12275

In [189]:
fname = '/Users/viralchitlangia/Documents/Songs_Audio_Trial/Training_Files/Beatles/WhileMyGuitarGentlyWeepsRemastered2009.wav'
with contextlib.closing(wave.open(fname,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / rate
    print((duration))

285.00172335600905


In [190]:
def duration(filename):
    with contextlib.closing(wave.open(filename,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / rate
        return int(duration)

In [191]:
def near_empty_array(x, n):
    y = [x]
    for i in range(0, n - 1):
        y.append(0)
    return y

near_empty_array(5, 4)

[5, 0, 0, 0]

In [192]:
if click == 0:
    X = []
    Y = []

In [193]:
if click == 0:
    for x in os.listdir("Training_Files"):
        if x != ".DS_Store":
            k = os.listdir("Training_Files/" + x)
            for j in tqdm(range(0, len(k))):
                if k[j] != ".DS_Store":
                    y, sr = librosa.load("Training_Files/" + x + "/" + k[j])
                    mfcc = librosa.feature.mfcc(y = y, sr = sr)
                    mfcc = np.resize(mfcc, (20, 10000))
                    time = duration("Training_Files/" + x + "/" + k[j])
                    row = near_empty_array(time, 10000)
                    mfcc = np.vstack([mfcc, row])
                    X.append(mfcc.tolist())
                    Y.append(x)

In [194]:
if click == 0:
    X = np.array(X)
    Y = np.array(Y)
    np.save("X.npy", X)
    np.save("Y.npy", Y)

In [195]:
np.unique(Y)

array(['Beach Boys', 'Beatles', 'Bob Dylan', 'Elvis Presley',
       'Led Zeppelin', 'Nirvana', 'Others', 'Pink Floyd', 'Queen'],
      dtype='<U13')

In [196]:
l = LabelEncoder()
l.fit(Y)
Y_t = l.transform(Y)

In [197]:
Y_t

array([1, 1, 1, ..., 2, 2, 2])

In [198]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_t, test_size=0.36)

In [199]:
X_train.shape

(909, 21, 10000)

In [200]:
model = models.Sequential()
model.add(layers.Conv1D(32, 2, activation='relu', input_shape=(21, 10000)))
#model.add(layers.AvgPool2D((2, 2)))
#model.add(layers.BatchNormalization(synchronized=True))
model.add(layers.Conv1D(64, 2, activation='relu'))
#model.add(layers.AvgPool1D(2))
model.add(layers.BatchNormalization(synchronized=True))
model.add(layers.Conv1D(128, 4, activation='relu'))
model.add(layers.AvgPool1D(2))
model.add(layers.BatchNormalization(synchronized=True))
model.add(layers.Conv1D(256, 3, activation='relu'))
model.add(layers.AvgPool1D(2))
model.add(layers.BatchNormalization(synchronized=True))
model.add(layers.Conv1D(512, 3, activation='relu'))
#model.add(layers.MaxPooling1D(2))
model.add(layers.BatchNormalization(synchronized=True))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(17, activation = 'softmax'))

In [201]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_50 (Conv1D)          (None, 20, 32)            640032    
                                                                 
 conv1d_51 (Conv1D)          (None, 19, 64)            4160      
                                                                 
 batch_normalization_42 (Bat  (None, 19, 64)           256       
 chNormalization)                                                
                                                                 
 conv1d_52 (Conv1D)          (None, 16, 128)           32896     
                                                                 
 average_pooling1d_14 (Avera  (None, 8, 128)           0         
 gePooling1D)                                                    
                                                                 
 batch_normalization_43 (Bat  (None, 8, 128)         

In [202]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-2),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=30, 
                    validation_data=(X_test, Y_test))

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


29/29 [==============================] - 7s 66ms/step - loss: 2.3627 - accuracy: 0.3036 - val_loss: 21.2656 - val_accuracy: 0.1191
Epoch 2/30
29/29 [==============================] - 1s 48ms/step - loss: 1.7374 - accuracy: 0.3982 - val_loss: 10.5309 - val_accuracy: 0.1211
Epoch 3/30
29/29 [==============================] - 1s 47ms/step - loss: 1.5830 - accuracy: 0.4323 - val_loss: 3.3592 - val_accuracy: 0.2539
Epoch 4/30
29/29 [==============================] - 1s 47ms/step - loss: 1.5329 - accuracy: 0.4389 - val_loss: 2.2658 - val_accuracy: 0.2930
Epoch 5/30
29/29 [==============================] - 1s 47ms/step - loss: 1.4743 - accuracy: 0.4642 - val_loss: 2.2602 - val_accuracy: 0.3320
Epoch 6/30
29/29 [==============================] - 1s 48ms/step - loss: 1.4383 - accuracy: 0.4587 - val_loss: 1.8822 - val_accuracy: 0.3789
Epoch 7/30
29/29 [==============================] - 1s 47ms/step - loss: 1.3703 - accuracy: 0.5050 - val_loss: 1.8027 - val_accuracy: 0.4199
Epoch 8/30
29/29 [====

In [203]:
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis = 1)

16/16 [==============================] - 1s 11ms/step


In [204]:
r2_score(Y_test, y_pred)

-0.22027676594582002